In [1]:
import nibabel as nib
import numpy as np 
import matplotlib.pyplot as plt
import os
import random
import torch
import pandas as pd
from tqdm.notebook import tqdm_notebook


In [2]:
import tarfile
file = tarfile.open('../input/brats-2021-task1/BraTS2021_Training_Data.tar')

file.extractall('./brain_images')
file.close()

In [ ]:
PATIENT_ID=input()
T1_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_t1.nii.gz"
T1CE_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_t1ce.nii.gz"
T2_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_t2.nii.gz"
FLAIR_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_flair.nii.gz"
MASK_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_seg.nii.gz"
img_t1=nib.load(T1_PATH)
img_t1ce=nib.load(T1CE_PATH)
img_t2=nib.load(T2_PATH)
img_flair=nib.load(FLAIR_PATH)
img_mask=nib.load(MASK_PATH)
image_data_t1=img_t1.get_fdata()
image_data_t1ce=img_t1ce.get_fdata()
image_data_t2=img_t2.get_fdata()
image_data_flair=img_flair.get_fdata()
mask=img_mask.get_fdata()
print(img_t1.header)
zero_matrix=np.zeros((240,240))
start=-1
end=1e9
nonEmptyMasks=[]
for i in range(0,155):
    if not np.array_equal(zero_matrix,mask[:,:,i]):
        if start==-1:
                start=i
        nonEmptyMasks.append(i)
        if i<end:
            end=i

def one_normal(x):
    if x==0:
        return 0
    else:
        return 1
plt.rcParams.update({'text.color': "red"})
LAYER=random.choice(nonEmptyMasks)
fig = plt.figure(figsize=(100,50))
rows = 5
cols=1
fig.add_subplot(rows, cols, 1)
cmask=np.vectorize(one_normal)(mask[:,:,LAYER])
plt.imshow(image_data_t1[:,:,LAYER]*cmask,cmap='gray')
plt.axis('off')
plt.title("T1")
fig.add_subplot(rows, cols,2)
plt.imshow(image_data_t1ce[:,:,LAYER]*cmask,cmap='gray')
plt.axis('off')
plt.title("T1CE")
fig.add_subplot(rows, cols, 3)
plt.imshow(image_data_t2[:,:,LAYER]*cmask,cmap='gray')
plt.axis('off')
plt.title("T2")
fig.add_subplot(rows, cols, 4)
plt.imshow(image_data_flair[:,:,LAYER]*cmask,cmap='gray')
plt.axis('off')
plt.title("FLAIR")
fig.add_subplot(rows, cols, 5)
plt.imshow(mask[:,:,LAYER],cmap='gray')
plt.axis('off')
plt.title("MASK")
fig.suptitle(f'Layer Number {LAYER}', fontsize=20)

In [4]:
labels=pd.read_csv('../input/mgmt-labels/train_labels_MGMT.csv')
CURRENT_FOLDER='./brain_images'
ALL_PATIENTS=[]
for f in tqdm_notebook(os.listdir(CURRENT_FOLDER),desc='Progress'):
    path_to_dir=os.path.join(CURRENT_FOLDER,f)
    if os.path.isdir(path_to_dir):
        ALL_PATIENTS.append(f)
ALL_MGMT_PATIENTS=[str(label).zfill(5) for label in labels['BraTS21ID']]
print("Total Number of patients : ",len(ALL_PATIENTS))
print("Total Number of patients with MGMT Data : ",len(ALL_MGMT_PATIENTS))
print("Total Number of patients with MGMT Methylation Positive : ",len(labels[labels['MGMT_value']==1]))
print("Total Number of patients with MGMT Methylation Negative : ",len(labels[labels['MGMT_value']==0]))

Progress:   0%|          | 0/1252 [00:00<?, ?it/s]

Total Number of patients :  1251
Total Number of patients with MGMT Data :  585
Total Number of patients with MGMT Methylation Positive :  307
Total Number of patients with MGMT Methylation Negative :  278


In [ ]:
model_resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
model_vgg = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=True)

In [13]:
for i in ALL_MGMT_PATIENTS:
    if f'BraTS2021_{i}' not in ALL_PATIENTS:
        print(i)
ALL_MGMT_PATIENTS=[i for i in ALL_MGMT_PATIENTS if f'BraTS2021_{i}' in ALL_PATIENTS]

00169
00197
00245
00308
00408
00564
00794
00998


In [115]:
MAX_T1=-1
MIN_T1=1e9
for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='T1'):
    img_t1=nib.load(f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_t1.nii.gz")
    image_data=img_t1.get_fdata()
    MAX_T1=max(MAX_T1,image_data.max())
    MIN_T1=min(MIN_T1,image_data.min())
print("Max Intensity in T1 Images : ",MAX_T1)
print("Min Intensity in T1 Images : ",MIN_T1)
MAX_T1CE=-1
MIN_T1CE=1e9
for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='T1CE'):
    img_t1=nib.load(f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_t1ce.nii.gz")
    image_data=img_t1.get_fdata()
    MAX_T1CE=max(MAX_T1CE,image_data.max())
    MIN_T1CE=min(MIN_T1CE,image_data.min())
print("Max Intensity in T1 Images : ",MAX_T1CE)
print("Min Intensity in T1 Images : ",MIN_T1CE)


T1:   0%|          | 0/577 [00:00<?, ?it/s]

Max Intensity in T1 Images :  12249.0
Min Intensity in T1 Images :  -162.0


T1CE:   0%|          | 0/577 [00:00<?, ?it/s]

Max Intensity in T1 Images :  24706.0
Min Intensity in T1 Images :  0.0


In [ ]:
MAX_T2=-1
MIN_T2=1e9
for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='T2'):
    img_t1=nib.load(f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_t2.nii.gz")
    image_data=img_t1.get_fdata()
    MAX_T2=max(MAX_T2,image_data.max())
    MIN_T2=min(MIN_T2,image_data.min())
print("Max Intensity in T2 Images : ",MAX_T2)
print("Min Intensity in T2 Images : ",MIN_T2)


In [ ]:
MAX_FLAIR=-1
MIN_FLAIR=1e9
for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='FLAIR'):
    img_t1=nib.load(f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_flair.nii.gz")
    image_data=img_t1.get_fdata()
    MAX_FLAIR=max(MAX_FLAIR,image_data.max())
    MIN_FLAIR=min(MIN_FLAIR,image_data.min())
print("Max Intensity in FLAIR Images : ",MAX_FLAIR)
print("Min Intensity in FLAIR Images : ",MIN_FLAIR)

In [ ]:
A=np.random.rand(240,240)*1000
print(A.max())
print(A.min())
A=Normalise(A,0,999)
print(A.max())
print(A.min())

In [50]:
ALL_DATA=[]
for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='Progress'):
    zero_matrix=np.zeros((240,240))
    start=-1
    end=1e9
    nonEmptyMasks=[]
    MASK_PATH=f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_seg.nii.gz"
    img_mask=nib.load(MASK_PATH)
    mask=img_mask.get_fdata()
    for j in range(0,155):
        if not np.array_equal(zero_matrix,mask[:,:,j]):
            ALL_DATA.append((i,j))
    
    

Progress:   0%|          | 0/577 [00:00<?, ?it/s]

In [51]:
len(ALL_DATA)

37302

In [153]:
import os
import pandas as pd
from torchvision.io import read_image

class MRI_Dataset(Dataset):
    def __init__(self,dataset, annotations_file, transform=None):
        self.dataset=dataset
        self.img_labels = pd.read_csv(annotations_file)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        _id,_slice=self.dataset[idx]
        img_path = f"./brain_images/BraTS2021_{str(_id).zfill(5)}/BraTS2021_{str(_id).zfill(5)}_t1.nii.gz"
        image = nib.load(img_path)
        image=image.get_fdata()[:,:,_slice]
        image = np.tile(image,(3,1,1))
        image=image.reshape(240,240,3)
        label = self.img_labels[self.img_labels['BraTS21ID']==int(_id)]['MGMT_value'].values[0]
        if self.transform:
            image = self.transform(image)
        return image, label

In [169]:
from torchvision import transforms, utils
class Normalise_range(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, t_min,t_max):

        self.t_min = t_min
        self.t_max = t_max

    def __call__(self, sample):
        image = sample
        ## Minmax Normalisation to 0 to 255
        image=image-self.t_min
        image=image/(self.t_max-self.t_min)
        image=image*255
        image[image>255.0]=255.0
        return image


In [171]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
train,test = train_test_split(ALL_DATA, test_size=0.33, random_state=42)
training_data=MRI_Dataset(train,'../input/mgmt-labels/train_labels_MGMT.csv',transform=transforms.Compose([
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
                                           ]))
test_data=MRI_Dataset(test,'../input/mgmt-labels/train_labels_MGMT.csv')
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
for i,j in train_dataloader:
    print(i.max(),j)
    break

tensor(33255.9733, dtype=torch.float64) tensor([0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0])
